In [3]:
import pandas as pd
import numpy as np
ventas = pd.read_csv("./data/retail.csv")

ventas.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,573744,21314,SMALL GLASS HEART TRINKET POT,8,2011-11-01 08:16:00,2.10,17733.0,United Kingdom
1,573744,21704,BAG 250g SWIRLY MARBLES,12,2011-11-01 08:16:00,0.85,17733.0,United Kingdom
2,573744,21791,VINTAGE HEADS AND TAILS CARD GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
3,573744,21892,TRADITIONAL WOODEN CATCH CUP GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
4,573744,21915,RED HARMONICA IN BOX,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom


In [41]:
df=pd.pivot_table(ventas, values='Quantity', index=['CustomerID'], columns=['Description'], aggfunc=np.sum,fill_value=0)
df

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,...,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12352.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12356.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12357.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12362.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18263.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18270.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,12,0,0,0,0,0
18274.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from scipy.sparse.linalg import svds

In [27]:
ratings =df.to_numpy(dtype='float64')
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
U,s, V = svds(ratings,k=10)

In [29]:
U.shape, s.shape, V.shape

((1711, 10), (10,), (10, 2728))

In [30]:
s_diag_matrix = np.zeros((s.shape[0], s.shape[0]))

for i in range(s.shape[0]):
    s_diag_matrix[i,i] = s[i]

In [32]:
s_diag_matrix.shape

(10, 10)

In [33]:
ratings_svd = U @ s_diag_matrix @ V

In [34]:
ratings_svd

array([[ 7.05918475e-04,  9.00046066e-03,  3.04685697e-03, ...,
         5.45435348e-03,  3.05631537e-05,  1.13244286e-04],
       [ 7.76428708e-05,  1.10636946e-03,  3.51628462e-04, ...,
         6.49120274e-04,  3.08282540e-06,  2.36906740e-05],
       [ 9.62473458e-06,  1.48313270e-04,  4.61095931e-05, ...,
         8.56126864e-05,  3.04357000e-07,  4.96870801e-06],
       ...,
       [ 1.95913167e-18,  1.53054101e-17,  2.09275195e-17, ...,
         1.25652811e-17,  5.84838511e-20,  6.11645301e-19],
       [-6.80403328e-08, -1.20138109e-06, -3.66484582e-07, ...,
        -7.14199078e-07, -2.15509241e-09, -4.59740035e-08],
       [ 2.07110519e-03,  3.16701300e-02,  1.03500775e-02, ...,
         1.88413349e-02,  6.89795187e-05,  1.00781653e-03]])

In [90]:
def recomendar(id_cliente, num_recomendaciones=5):
    id=list(zip(df.index, range(len(df.index))))
    for i in id:
        if i[0]== id_cliente:
            x=0
            lista=[]
            for producto in df.loc[id_cliente]:
                if producto != 0:
                    lista.append(x)
                x=x+1
            ratings_nuevos=list( zip (ratings_svd[i[1]], range(len(ratings_svd[i[1]]))))
            for numero in lista:
                ratings_nuevos.pop(numero)
            recomendaciones_ordenadas= sorted(ratings_nuevos, key=lambda x: x[0], reverse=True)
            reco=recomendaciones_ordenadas[:num_recomendaciones]
            recomendaciones=[]
            for articulo in reco:
                recomendaciones.append(df.columns[articulo[1]])
    return recomendaciones

In [92]:
recomendar(12352.0,10)

['MINI PAINT SET VINTAGE ',
 'JUMBO BAG RED RETROSPOT',
 'PLACE SETTING WHITE STAR',
 'CHRISTMAS HANGING STAR WITH BELL',
 'CHARLOTTE BAG PINK POLKADOT',
 'RAIN PONCHO RETROSPOT',
 '12 PENCIL SMALL TUBE WOODLAND',
 'LUNCH BAG PAISLEY PARK  ',
 'SPACEBOY LUNCH BOX ',
 'LUNCH BAG SPACEBOY DESIGN ']